Define some filenames

In [ ]:
import os
data_filename   = os.path.join(os.pardir, "data", "all_sz.json")
params_filename = os.path.join(os.pardir, "data", "default_params.json")

Load the subduction zone parameters

In [ ]:
import json
with open(data_filename, "r") as fp:
    szparams = json.load(fp)

The "minimum" resolution, used to scale resolutions throughout the mesh.  The current default of 10.0km is way too high for a real subduction zone model but helps to keep this test notebook fast and readable.

In [ ]:
minres = 10.0 # km

In [ ]:
import numpy as np

Just for reference this is the code I used to generate the default parameters json file but it's turned off now and we just read it in in the next cell

Load the default parameters

In [ ]:
with open(params_filename, "r") as fp:
    default_params = json.load(fp)

List the available subduction zones that we've just loaded from the json file

In [ ]:
print("Available subduction zones:")
for k in szparams.keys(): print(k)

## This is where you can switch between subduction zones!!

Choose a subduction zone from the list on the first line of the following cell to load the geometrical parameters from the dictionary

In [ ]:
szname = "07_Nicaragua"  # change this to switch subduction zones
szdata = szparams[szname]
coast_distance = szdata['deltaxcoast']
extra_width = szdata['deltaxwidth']
lc_depth = szdata['deltazmoho']
uc_depth = szdata['deltazuc']
io_depth = szdata['deltazoutflow']
points = np.asarray([szdata['trench']]+szdata['slabpoints'])
xs = points[:,0]
ys = points[:,1]
res = [1*minres if y > -200.0 else 3*minres for y in ys]

Load the geometry module so we can build a mesh based on the above parameters

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))
import geometry as geo

Define the slab spline

In [ ]:
slab = geo.SlabSpline(xs, ys, res=res, sid=default_params['slab_sid'], name="Slab")

Define the subduction geometry, including crustal layers and plot

In [ ]:
geom = geo.SubductionGeometry(slab, **default_params, 
                              wedge_top_res=default_params["wedge_top_res_fact"]*minres,
                              slab_base_res=default_params["slab_base_res_fact"]*minres,
                              coast_distance=coast_distance, extra_width=extra_width)
# adding the coupling depths may or may not be necessary
# depending on if they were included in the slab spline data already or not
# the geometry class should ignore them if they aren't necessary
slab.addpoint(80, "Slab::PartialCouplingDepth", res=1*minres)
slab.addpoint(82.5, "Slab::FullCouplingDepth", res=1*minres)
# add a lower crust
geom.addcrustlayer(lc_depth, "LowerCrust", 
                   sid=default_params["lc_base_sid"], rid=default_params["lc_rid"],
                   slab_res=default_params["lc_slab_res_fact"]*minres,
                   side_res=default_params["lc_side_res_fact"]*minres)
# oceanic subduction zones don't define an upper crust
if uc_depth is not None:
    geom.addcrustlayer(uc_depth, "UpperCrust", 
                       sid=default_params["uc_base_sid"], rid=default_params["uc_rid"],
                       slab_res=default_params["uc_slab_res_fact"]*minres,
                       side_res=default_params["uc_side_res_fact"]*minres)
# add the pre-defined in-out point
geom.addwedgesidepoint(io_depth, "WedgeSide::InOut", line_name="UpperWedgeSide", 
                       res=default_params["io_depth_res_fact"]*minres, 
                       sid=default_params["upper_wedge_side_sid"])
# add the slab detector point
geom.addslabpoint(default_params["slab_det_depth"], "Slab::DetectorPoint", res=1*minres)
geom.plot()